# Image tiling for annotation

#### Meanings of arguments
- ```-ratioheight``` : proportion of tile  w.r.t height of image. Example 0.5 means dividing the image in two bands w.r.t height.
- ```-ratiowidth``` : proportion of tile w.r.t to width of image. Example 1.0 means the width of the tile is the same as the image.
- ```-overlapfactor``` : percentage of overlap. It should be less than 1.
- ```-rmheight``` : percentage of height to remove or crop at bottom and top
- ```-rmwidth``` : percentage of width to remove or crop on each side of the image
- ```-pattern``` : "**/*.JPG" will get all .JPG images in directory and subdirectories. On windows it will get both .JPG and .jpg. On unix it will only get .JPG images


In [ ]:
# New script for tiling data
# images_to_tile = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images"
# destination_directory = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images-tiled"
!python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Dry season\Kapiri\Camp 6-8\Rep 2" 0 0 0 -overlapfactor 0.1  -ratiowidth 0.33334 -ratioheight 0.5 -rmheight 0.21 -rmwidth 0.1 -dest "D:\PhD\Data per camp\Dry season\Kapiri\Camp 6-8\Rep 2 - tiled" -pattern "**/*.JPG"

# Pre-annotating data for Labelstudio

In [1]:
from datalabeling.annotator import Annotator
import os
from pathlib import Path

### Creating a JSON file to be uuploaded to Label studio

In [ ]:
# Example
# provide correct alias, "pt", "onnx"
alias = "last" # the aliases are found in mlflow tracker UI
handler = Annotator(mlflow_model_alias=alias,
                    # dotenv_path="../.env"
                    )
path_img_dir=r"D:\PhD\Data per camp\Dry season\Kapiri\Camp 6-8\Rep 2 - tiled"
root="D:\\"
save_json_path = os.path.join(Path(path_img_dir).parent,
                              f"{Path(path_img_dir).name}_preannotation_label-studio.json")

# build and saves json
directory_preds = handler.build_upload_json(path_img_dir=path_img_dir,
                                            root=root,
                                            save_json_path=save_json_path,
                                            pattern="**/*.JPG")

### Pre-annotating an existing project using Label studio API
It seems that it will not work well (i.e. filtering) with older projects created prior to Label studio software update.
It is the **recommended way of pre-annotating data in Labelstudio**.

In [ ]:
# provide correct alias, "pt", "onnx"
alias = "last"
handler = Annotator(mlflow_model_alias=alias,
                    dotenv_path="../.env")
project_id = ... # insert correct project_id by loooking at the url
handler.upload_predictions(project_id=project_id)

In [6]:
from label_studio_ml.utils import get_local_path
from pathlib import Path

In [ ]:
get_local_path("/data/local-files/?d=PhD%5CData%20per%20camp%5CDry%20season%5CLeopard%20rock%5CCamp%2022%20%2B%2037-40%5CRep%201%20-%20tiled%5CDJI_20231002112756_0001_0.JPG")

# Inference with Sahi

In [1]:
# from ultralytics import YOLO
from sahi.predict import get_sliced_prediction
# import torch
from PIL import Image
from time import time
from sahi.models.yolov8 import Yolov8DetectionModel
from datalabeling.annotator import Yolov8ObbDetectionModel

c:\Users\fadel\miniconda3\envs\datalabeling\Lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
sahi_model_obb = Yolov8ObbDetectionModel(model_path=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\base_models_weights\yolov8-wildai-obb.pt")
sahi_model = Yolov8DetectionModel(model_path=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\base_models_weights\yolov8.kaza.pt")
image_path = r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\images\01f1653a94f14044bf11d78c5b4221d1.JPG"
image = Image.open(image_path)
print(image.size)

(4000, 3000)


In [3]:
result = get_sliced_prediction(image, 
                                sahi_model_obb,
                                slice_height=1280,
                                slice_width=1280,
                                overlap_height_ratio=0.1,
                                overlap_width_ratio=0.1,
                                postprocess_type='NMS',
                            ) 

Performing prediction on 12 slices.


In [4]:
result.export_visuals('../.tmp')

In [ ]:
image_path = r"D:\PhD\Data per camp\Dry season\Kapiri\Camp 6-8\Rep 1 - tiled\DJI_20231003081043_0016_1.JPG"

In [ ]:
# plt.imshow(tile)
# plt.show()

# YOLO data_config.yaml 

In [ ]:
import yaml
import json
from arguments import Arguments

In [ ]:
# load yaml
with open(r"D:\PhD\Data per camp\IdentificationDataset\data_config.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)
yolo_config

In [ ]:
# load label mapping
args = Arguments()
with open(r"D:\PhD\Data per camp\IdentificationDataset\label_mapping.json",'r') as file:
    label_map = json.load(file)
names = [p['name'] for p in label_map if p['name'] not in args.discard_labels ]
label_map = dict(zip(range(len(names)),names))
label_map

In [ ]:
yolo_config.update({'names':label_map,'nc':len(label_map)})
yolo_config

In [ ]:
with open(r"D:\PhD\Data per camp\IdentificationDataset\data_config.yaml",'w') as file:
    yaml.dump(yolo_config,file,default_flow_style=False, sort_keys=False)

# Dataset distribution

In [ ]:
import yaml
import pandas as pd
import os
from pathlib import Path

In [ ]:
# load yaml
with open(r"D:\PhD\Data per camp\Extra training data\WAID\data_config.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)
yolo_config

In [ ]:
label_map = yolo_config['names']

In [ ]:
split = 'train'

path_dataset = os.path.join(yolo_config['path'],yolo_config[split][0])
path_dataset = path_dataset.replace('images','labels')

path_dataset

In [ ]:
labels = list()

for txtfile in Path(path_dataset).glob("*.txt"):

    df = pd.read_csv(txtfile,sep=" ",names = ['class','x','y','w','h'] )
    df['class'] = df['class'].astype(int)    
    df['image'] = txtfile.stem
    labels.append(df)


In [ ]:
df = pd.concat(labels,axis=0)
df['class'] = df['class'].map(label_map)

In [ ]:
images_per_class = dict()
for cls in df['class'].unique():
    num_imge = df.loc[df['class'] == cls,'image'].unique().shape[0]
    images_per_class[cls] = num_imge

In [ ]:
print("Split:", split)
print(images_per_class)

In [ ]:
print('Split:',split)
print(df['class'].value_counts())

In [ ]:
df['class'].value_counts().plot(kind='bar',figsize=(10,5),logy=True,title=f"{split} label distribution")

# Computing metrics on Validation set

In [40]:
from ultralytics import YOLO
# from pathlib import Path
import torch

In [35]:
# Load a model
path = r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\base_models_weights\yolov8-wildai-obb.pt"
# path = r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\base_models_weights\yolov5su.pt"
model = YOLO(path)  

In [36]:
pred = model.predict(r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\images\01f1653a94f14044bf11d78c5b4221d1.JPG")


image 1/1 C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\images\01f1653a94f14044bf11d78c5b4221d1.JPG: 480x640 1815.5ms
Speed: 17.0ms preprocess, 1815.5ms inference, 9.4ms postprocess per image at shape (1, 3, 480, 640)


In [38]:
[result.obb for result in pred]

[ultralytics.engine.results.OBB object with attributes:
 
 cls: tensor([0., 0., 0., 0.])
 conf: tensor([0.6630, 0.3514, 0.2796, 0.2695])
 data: tensor([[6.8862e+02, 1.2766e+03, 8.7202e+01, 8.0733e+01, 1.4624e-01, 6.6303e-01, 0.0000e+00],
         [2.7321e+02, 2.2809e+03, 7.7099e+01, 6.9344e+01, 1.9605e+00, 3.5140e-01, 0.0000e+00],
         [6.2843e+02, 1.4832e+03, 7.9159e+01, 5.8002e+01, 4.8534e-01, 2.7962e-01, 0.0000e+00],
         [3.5846e+02, 1.2539e+03, 7.7243e+01, 6.7475e+01, 5.9167e-01, 2.6951e-01, 0.0000e+00]])
 id: None
 is_track: False
 orig_shape: (3000, 4000)
 shape: torch.Size([4, 7])
 xywhr: tensor([[6.8862e+02, 1.2766e+03, 8.7202e+01, 8.0733e+01, 1.4624e-01],
         [2.7321e+02, 2.2809e+03, 7.7099e+01, 6.9344e+01, 1.9605e+00],
         [6.2843e+02, 1.4832e+03, 7.9159e+01, 5.8002e+01, 4.8534e-01],
         [3.5846e+02, 1.2539e+03, 7.7243e+01, 6.7475e+01, 5.9167e-01]])
 xyxy: tensor([[ 639.5987, 1230.2849,  737.6347, 1322.8635],
         [ 226.4875, 2232.0591,  319.9226, 

In [47]:
pred[0].obb.xyxy

tensor([[ 639.5987, 1230.2849,  737.6347, 1322.8635],
        [ 226.4875, 2232.0591,  319.9226, 2329.7217],
        [ 579.8905, 1439.1091,  676.9661, 1527.3413],
        [ 307.5852, 1204.3851,  409.3313, 1303.4718]])

In [48]:
pred[0].obb.cls

tensor([0., 0., 0., 0.])

In [49]:
pred[0].obb.conf

tensor([0.6630, 0.3514, 0.2796, 0.2695])

In [ ]:
# Customize validation settings
validation_results = model.val(data=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\data_config.yaml",
                                imgsz=640,
                                batch=8,
                                conf=0.25,
                                iou=0.5,
                                device="cpu")

In [ ]:
# Compute predictions
from datalabeling.annotator import Detector

handler = Detector(path_to_weights=path,confidence_threshold=0.3)
predictions = handler.predict_directory(r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\images")